# Introduction

> The script automates gathering of tweet data from the archives

## Packages

In [218]:
#import packages 
import pandas as pd 
import numpy as np
import os 
import time
import requests
import itertools    
import json
import re

from datetime import date
from datetime import datetime
from datetime import timedelta

#import data scraping packages 
import tweepy
# from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import Cursor
from tweepy import API

#import web scrapping packages 
import bs4 as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from selenium_stealth import stealth
import undetected_chromedriver as uc



#sns module 
import snscrape.modules.twitter as sntwitter
import snscrape.modules.facebook as snfacebook
import snscrape.modules.instagram as sninstagram

#postgresql database packages 
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine

import csv
from io import StringIO

#check the start time
start_time = datetime.now()

## Extract Time

In [232]:
#generate the 
# expected_days =365*1
number_of_dates = 365*0.2
end_date = date.today() #-timedelta(days=expected_days)
start_date =  end_date - timedelta(days=number_of_dates)
duration = end_date - start_date

print()
print()
print(f"The start date: {start_date} \n end date is: {end_date} and \n time duration is: {duration}")



The start date: 2022-06-27 
 end date is: 2022-09-08 and 
 time duration is: 73 days, 0:00:00


In [220]:
#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%B %d %Y %H %M %S")

#print date 
print()
print()
print("Todays date: ", formated_date)



Todays date:  September 08 2022 09 37 53


### Keywords

> The following are search keywords for cyber security concerns 

In [221]:
#define variables 
keywords_list = ["Trojan", "Spyware", "ransomware", "Botnet", "Defense", "Evasion","ingress tool transfer", "Malicious link","firmware", "Bruteforce", "automated exfiltration", 
                 "DNSSEC traffic", "channel attacks", "DHCP spoofing","Protocol impersonation", "Denial of Service", "Phishing", "Spearphishing", "OS Credential Dumping"]

#search words 
#fornat the sland 
print()
print()
print(f"{'*'*50} Keyword list {'*'*60}")
print(keywords_list)

print()
print()
#search words 
print(f"{'*'*50} Joined Keyword List {'*'*50}")
search_word = "|".join([j.lower() for j in keywords_list])
print(search_word)





************************************************** Keyword list ************************************************************
['Trojan', 'Spyware', 'ransomware', 'Botnet', 'Defense', 'Evasion', 'ingress tool transfer', 'Malicious link', 'firmware', 'Bruteforce', 'automated exfiltration', 'DNSSEC traffic', 'channel attacks', 'DHCP spoofing', 'Protocol impersonation', 'Denial of Service', 'Phishing', 'Spearphishing', 'OS Credential Dumping']


************************************************** Joined Keyword List **************************************************
trojan|spyware|ransomware|botnet|defense|evasion|ingress tool transfer|malicious link|firmware|bruteforce|automated exfiltration|dnssec traffic|channel attacks|dhcp spoofing|protocol impersonation|denial of service|phishing|spearphishing|os credential dumping


### African Countries

In [222]:
#countries involved
name = ['Algeria','Angola','Benin','Botswana','Burkina_Faso','Burundi','Cameroon','Cape_Verde',
             'Central_African_Republic','Chad','Comoros','Democratic_Republic_of_the_Congo','Djibouti','Egypt',
             'Equatorial_Guinea',' Eritrea',' Ethiopia',' Gabon',' Gambia',' Ghana','Guinea',' Guinea-Bissau',
             'Ivory_Coast','Kenya',' Lesotho',' Liberia',' Libya',' Madagascar',' Malawi',' Mali',' Mauritania','Mauritius',
             'Morocco',' Mozambique',' Namibia',' Niger',' Nigeria','Republic of the Congo',' Rwanda','Sao_Tome_and_Principe',
             'Senegal',' Seychelles',' Sierra_Leone',' Somalia',' South_Africa',' Sudan','South_Sudan','Swaziland',' Tanzania',' Togo',
             'Tunisia',' Uganda',' Zambia',' Zimbabwe']

# #strip any white spaces 
country_name = [x.strip().lower().replace('_','-').replace(' ','-') for x in name]

#search words 
countries = "|".join([j.strip().lower() for j in name])

In [223]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [224]:
 driver = webdriver.Chrome(r'C:\Users\chromedriver_win32\chromedriver.exe')

C:\Users\moses\AppData\Local\Temp\ipykernel_15716\1516475625.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\chromedriver_win32\chromedriver.exe')


## Place Specific 

> Countries towns and cities co-ordinates [link](https://www.countrycoordinate.com/)

In [8]:
# #get the data in a dataframe 
# coordinates_df =  pd.DataFrame()
# count = 0

# for country in country_name:
#     #get the url
#     url = f"https://www.countrycoordinate.com/country-{country}/"
#     count +=1
#     print(f"Num: {count}; link: {url}") 
#     try:
#         driver = uc.Chrome(use_subprocess=True)
#         driver.get(url)
#         time.sleep(5)
#         soup = bs.BeautifulSoup(driver.page_source,'lxml')

#         # get data table & save as dataframe
#         table_val = soup.find("table",{'table table-bordered table-hover'})
#         table_val_df = pd.read_html(str(table_val))[0]

#         table_val_df.columns = ['city', 'coords']
#         table_val_df['country'] = country
#         table_val_df['url'] = url
#         table_val_df[['lat', 'lon']] = table_val_df['coords'].str.split('/', expand = True)
#         table_val_df.drop(columns = ['coords'], inplace = True)
#         if table_val_df.shape[0]>1:
#             coordinates_df = pd.concat([coordinates_df,table_val_df])
#         else:
#             pass


#         #get the number of other pages 
#         page_num_table = soup.find_all("table", {"class":"table table-bordered centered"})



#           #loop through other pages
#         print(f"{'*'*15} base page done {'*'*15}")
#         print(f"{'*'*15} Other pages {'*'*15}")
#         for tab in page_num_table:
#             row = tab.find_all('a')
#             row = [i.text.strip() for i in row]
#             row = row[:-1]
#             print(f"{'*'*15} pages to extract {len(row)+1} {'*'*15}")

#             for page in row: 
#                 next_page_url = url+f"?page={page}"
#                 print(next_page_url)

#                 #get the data 
#                 driver.get(next_page_url)
#                 time.sleep(2)
#                 soup_page = bs.BeautifulSoup(driver.page_source, 'lxml')

#                 # get data table & save as dataframe
#                 table_val_next = soup_page.find("table",{'table table-bordered table-hover'})
#                 table_val_df_1 = pd.read_html(str(table_val_next))[0]
#                 table_val_df_1.columns = ['city', 'coords']
#                 table_val_df_1['country'] = country
#                 table_val_df_1['url'] = url
#                 table_val_df_1[['lat', 'lon']] = table_val_df_1['coords'].str.split('/', expand = True)
#                 table_val_df_1.drop(columns = ['coords'], inplace = True)
#                 if table_val_df_1.shape[0]>1:
#                     coordinates_df = pd.concat([coordinates_df,table_val_df_1])
#                 else:
#                     pass
                
#     except Exception as e:
#         print(f'The error is: {str(e)}')
#         pass
    
# #clean the data 

# #reset the index
# coordinates_df.reset_index(drop = True, inplace = True)

# #clean the data 
# coordinates_df['city'] = coordinates_df['city'].str.split('(', expand = True)[0]

# #drop duplicate towns
# print(f"Rows before droping duplicates {coordinates_df.shape[0]}")
# coordinates_df.drop_duplicates(subset = ['city', 'country'], inplace = True, keep = 'last')
# print(f"Rows befor droping duplicates {coordinates_df.shape[0]}")

# #print the sample 
# coordinates_df.sample(3)

coordinates_df = pd.

Num: 1; link: https://www.countrycoordinate.com/country-algeria/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 7 ***************
https://www.countrycoordinate.com/country-algeria/?page=2
https://www.countrycoordinate.com/country-algeria/?page=3
https://www.countrycoordinate.com/country-algeria/?page=4
https://www.countrycoordinate.com/country-algeria/?page=5
https://www.countrycoordinate.com/country-algeria/?page=12
https://www.countrycoordinate.com/country-algeria/?page=13
Num: 2; link: https://www.countrycoordinate.com/country-angola/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 2 ***************
https://www.countrycoordinate.com/country-angola/?page=2
Num: 3; link: https://www.countrycoordinate.com/country-benin/
*************** base page done ***************
*************** Other pages ***************
*************** pages to e

*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 3 ***************
https://www.countrycoordinate.com/country-libya/?page=2
https://www.countrycoordinate.com/country-libya/?page=3
Num: 28; link: https://www.countrycoordinate.com/country-madagascar/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 6 ***************
https://www.countrycoordinate.com/country-madagascar/?page=2
https://www.countrycoordinate.com/country-madagascar/?page=3
https://www.countrycoordinate.com/country-madagascar/?page=4
https://www.countrycoordinate.com/country-madagascar/?page=5
https://www.countrycoordinate.com/country-madagascar/?page=6
Num: 29; link: https://www.countrycoordinate.com/country-malawi/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 2 ***************
https://www.countrycoor

Num: 52; link: https://www.countrycoordinate.com/country-uganda/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 4 ***************
https://www.countrycoordinate.com/country-uganda/?page=2
https://www.countrycoordinate.com/country-uganda/?page=3
https://www.countrycoordinate.com/country-uganda/?page=4
Num: 53; link: https://www.countrycoordinate.com/country-zambia/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 4 ***************
https://www.countrycoordinate.com/country-zambia/?page=2
https://www.countrycoordinate.com/country-zambia/?page=3
https://www.countrycoordinate.com/country-zambia/?page=4
Num: 54; link: https://www.countrycoordinate.com/country-zimbabwe/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 3 ***************
https://www.countrycoordinate.com/c

,city,country,url,lat,lon
1915,Takamaka,seychelles,https://www.countrycoordinate.com/country-seyc...,-4.784369,55.508101
714,Makadi Bay,egypt,https://www.countrycoordinate.com/country-egypt/,26.984847,33.902051
481,Boali,central-african-republic,https://www.countrycoordinate.com/country-cent...,4.888774,18.142867


In [225]:
country_df = coordinates_df[coordinates_df['country'] == 'kenya']
# country_df = coordinates_df.copy()

## Extract tweets using keywords

In [301]:
#create and empty df
df_x = pd.DataFrame()

#get the present da

#define the max keywords
maxcounts = 2
maxcounts_v2 = 0
count = 0
multiplyer = 5

try: 
    for row in country_df.index[:1]:
            country = country_df['country'][row]
            nearby_location = country_df['city'][row]
            lat = country_df['lat'][row]
            lon = country_df['lon'][row]
            circumfrence = "10km"
            coor = lat+","+lon+","+circumfrence
            coor = format(coor)
            print()
            print(f"country: {country}, nearby_loc: {nearby_location},  coordinates: {lat+', '+lon}, radius: {circumfrence}")
            
            try:
                for word in keywords_list[:2]:
                    count +=1
                    
                    print(f"{'*'*20} instance within: {count};  word: {word} {'*'*20}")
                    scraped_tweets = sntwitter.TwitterSearchScraper(f'{str(word)} since:{start_date} until:{end_date} geocode:"{coor}"').get_items()

                    sliced_scraped_tweets = itertools.islice(scraped_tweets, maxcounts)  
                    df_coord = pd.DataFrame(sliced_scraped_tweets)
                    df_coord.columns = df_coord.columns.str.lower()
                    df_coord.reset_index(drop = True, inplace = True)
                    df_coord['date'] = df_coord['date'].apply(lambda x: x.strftime("%d-%m-%Y"))
                    df_coord['date'] = df_coord['date'].apply(lambda x: datetime.strptime(x, "%d-%m-%Y"))
                    df_coord['keyword'] = word
                    df_coord['nearby_location'] = nearby_location
                    df_coord['country'] = country
                    
                    #get the new date 
                    new_date = df_coord['date'][0]
                    
                    #update the dataset
                    df_x = pd.concat([df_x, df_coord])
                        
                    #print the progress of the loop
                    print(' '*120)
                    print(f" {'> '*10} tweet id: {df_coord['id'][0]}, tweet date: {df_coord['date'][0]}  {' <'*10} ")
            
            
                    
                    #check if the start date is greater
                    while new_date > start_date:
                        
                        maxcounts_v2+=1
                        scraped_tweets = sntwitter.TwitterSearchScraper(f'{str(word)} since:{start_date} until:{new_date.date()} geocode:"{coor}"').get_items()

                        sliced_scraped_tweets = itertools.islice(scraped_tweets, maxcounts_v2)  
                        df_coord = pd.DataFrame(sliced_scraped_tweets)
                        df_coord.columns = df_coord.columns.str.lower()
                        df_coord['date'] = df_coord['date'].apply(lambda x: x.strftime("%d-%m-%Y"))
                        df_coord['date'] = df_coord['date'].apply(lambda x: datetime.strptime(x, "%d-%m-%Y"))
                        df_coord['keyword'] = word
                        df_coord['nearby_location'] = nearby_location
                        df_coord['country'] = country
                        
                        #update the dataset if data is not present in the intia list
                        df_x = pd.concat([df_x[~df_x['id'].isin(df_coord['id'])], df_coord])

                        print(' '*120)                                              
                        print(f"{' >'*20}  dynamic count: {maxcounts_v2}  {'< '*20} ")
                        
                        #break the loop incase its infinate
                        if maxcounts_v2 > 5:
                            break
                        
                    #restore dynamic count to zero 
                    maxcounts_v2 = 0
                        
                        
                        
                    
            except Exception as e:
                print(f'The error was: {str(e)}')
except Exception as e:
    print(f'The error was: {str(e)}')

#clean the d
df_x.reset_index(drop = True, inplace = True)
df_x


country: kenya, nearby_loc: Nairobi,  coordinates: -1.292066 ,  36.821946, radius: 10km
******************** instance within: 1;  word: Trojan ********************
                                                                                                                        
 > > > > > > > > > >  tweet id: 1567440630095265793, tweet date: 2022-09-07 00:00:00   < < < < < < < < < < 


C:\Users\moses\AppData\Local\Temp\ipykernel_15716\3823635575.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while new_date > start_date:


                                                                                                                        
 > > > > > > > > > > > > > > > > > > > >  dynamic count: 1  < < < < < < < < < < < < < < < < < < < <  


C:\Users\moses\AppData\Local\Temp\ipykernel_15716\3823635575.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while new_date > start_date:


                                                                                                                        
 > > > > > > > > > > > > > > > > > > > >  dynamic count: 2  < < < < < < < < < < < < < < < < < < < <  


...

C:\Users\moses\AppData\Local\Temp\ipykernel_15716\3823635575.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while new_date > start_date:


                                                                                                                        
 > > > > > > > > > > > > > > > > > > > >  dynamic count: 5  < < < < < < < < < < < < < < < < < < < <  


C:\Users\moses\AppData\Local\Temp\ipykernel_15716\3823635575.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while new_date > start_date:


                                                                                                                        
 > > > > > > > > > > > > > > > > > > > >  dynamic count: 6  < < < < < < < < < < < < < < < < < < < <  


,url,date,content,renderedcontent,id,user,replycount,retweetcount,likecount,quotecount,...,inreplytotweetid,inreplytouser,mentionedusers,coordinates,place,hashtags,cashtags,keyword,nearby_location,country
0,https://twitter.com/MoffatAkumbi/status/156744...,2022-09-07,@trojan_ragdde @ledamalekina You missed the po...,@trojan_ragdde @ledamalekina You missed the po...,1567440630095265793,"{'username': 'MoffatAkumbi', 'id': 79669048447...",0,0,0,0,...,1567440288691412992.0,"{'username': 'trojan_ragdde', 'id': 1425254811...","[{'username': 'trojan_ragdde', 'id': 142525481...",None,None,None,None,Trojan,Nairobi,kenya
1,https://twitter.com/Lemeiyan/status/1567214837...,2022-09-06,@trojan_ragdde @Kenyans Wewe kacheze huko bana.,@trojan_ragdde @Kenyans Wewe kacheze huko bana.,1567214837343698946,"{'username': 'Lemeiyan', 'id': 901558773864681...",1,0,0,0,...,1567214388431540224.0,"{'username': 'trojan_ragdde', 'id': 1425254811...","[{'username': 'trojan_ragdde', 'id': 142525481...",None,None,None,None,Trojan,Nairobi,kenya
2,https://twitter.com/TToryquer/status/156707438...,2022-09-06,@MissTichie @MarthaKarua Martha was a Trojan H...,@MissTichie @MarthaKarua Martha was a Trojan H...,1567074381938974725,"{'username': 'TToryquer', 'id': 14557737314234...",0,0,0,0,...,1567013111558709248.0,"{'username': 'MissTichie', 'id': 1364457704, '...","[{'username': 'MissTichie', 'id': 1364457704, ...",None,None,None,None,Trojan,Nairobi,kenya
3,https://twitter.com/Edgerayier/status/15668321...,2022-09-05,@citizentvkenya Trojan horse,@citizentvkenya Trojan horse,1566832183565815812,"{'username': 'Edgerayier', 'id': 173788176, 'd...",0,0,1,0,...,1566826322306994176.0,"{'username': 'citizentvkenya', 'id': 70394965,...","[{'username': 'citizentvkenya', 'id': 70394965...",None,None,None,None,Trojan,Nairobi,kenya
4,https://twitter.com/PolycarpHinga/status/15667...,2022-09-05,"Now reading conspiracy theories like\n""Uhuru k...","Now reading conspiracy theories like\n""Uhuru k...",1566773694327918596,"{'username': 'PolycarpHinga', 'id': 480295789,...",9,7,56,3,...,NaN,None,None,None,None,None,None,Trojan,Nairobi,kenya
5,https://twitter.com/chepkotkotbore/status/1566...,2022-09-05,@nyar_asego @WMMartha Supposing he was a Troja...,@nyar_asego @WMMartha Supposing he was a Troja...,1566772313508483072,"{'username': 'chepkotkotbore', 'id': 260407108...",0,0,0,0,...,1566746443116863488.0,"{'username': 'nyar_asego', 'id': 1351275549201...","[{'username': 'nyar_asego', 'id': 135127554920...",None,None,None,None,Trojan,Nairobi,kenya
6,https://twitter.com/kimondo_paul/status/156674...,2022-09-05,@WehliyeMohamed @RailaOdinga @WilliamsRuto Uhu...,@WehliyeMohamed @RailaOdinga @WilliamsRuto Uhu...,1566746899796877313,"{'username': 'kimondo_paul', 'id': 1876370436,...",0,0,2,0,...,1566739013146378240.0,"{'username': 'WehliyeMohamed', 'id': 257905607...","[{'username': 'WehliyeMohamed', 'id': 25790560...",None,None,None,None,Trojan,Nairobi,kenya
7,https://twitter.com/Tintoidrissou/status/15670...,2022-09-06,Nephew of jailed Hotel Rwanda dissident hacked...,Nephew of jailed Hotel Rwanda dissident hacked...,1567055107488972802,"{'username': 'Tintoidrissou', 'id': 217544132,...",0,0,0,0,...,None,None,None,None,None,None,None,Spyware,Nairobi,kenya
8,https://twitter.com/oneike/status/156637638438...,2022-09-04,namimi sjamaliza kusomea the spyware that dele...,namimi sjamaliza kusomea the spyware that dele...,1566376384389386243,"{'username': 'oneike', 'id': 555163828, 'displ...",0,0,0,0,...,None,None,None,None,None,None,None,Spyware,Nairobi,kenya
9,https://twitter.com/EADataHandlers/status/1564...,2022-08-30,How do hackers Hack? Hackers use malware to st...,How do hackers Hack? Hackers use malware to st...,1564549217263165440,"{'username': 'EADataHandlers', 'id': 531086334...",0,0,2,0,...,None,None,None,None,None,"[DataRecovery, cybersecurity]",None,Spyware,Nairobi,kenya


## Clean the extracted tweets

In [303]:

#subset the data
df = df_x[['id', 'date', 'content', 'renderedcontent', 'user', 'replycount','retweetcount', 'likecount', 'quotecount',
           'url', 'lang', 'retweetedtweet', 'quotedtweet','mentionedusers', 'coordinates', 'place', 'hashtags', 'keyword', 
           'nearby_location', 'country']]

#format the date
df['date'] = df['date'].apply(lambda x: x.strftime("%d-%b-%Y %H:%M:%S"))

# getting user's location
df.reset_index(drop = True, inplace = True)
df['username'] = df['user'].apply(lambda x: x['username'])
df['displayname'] = df['user'].apply(lambda x: x['displayname'])
df['user_id'] = df['user'].apply(lambda x: x['id'])
df['user_location'] = df['user'].apply(lambda x: x['location'])
df['description'] = df['user'].apply(lambda x: x['description'])
df['followersCount'] = df['user'].apply(lambda x: x['followersCount'])
df['friendsCount'] = df['user'].apply(lambda x: x['friendsCount'])
df['statusesCount'] = df['user'].apply(lambda x: x['statusesCount'])
df['favouritesCount'] = df['user'].apply(lambda x: x['favouritesCount'])
df['protected'] = df['user'].apply(lambda x: x['protected'])

#mentions 
pattern_1 = r'#(\w+)'
pattern_2 = r'@(\w+)'
# df['hashtag'] = df['content'].apply(lambda x: re.findall(pattern_1, x))
df['mentions'] = df['content'].apply(lambda x: re.findall(pattern_2, x))

#drop some columns 
df.drop(columns = ['user', 'mentionedusers'], inplace = True)


#clean the data 
##lower the text
df['content'] = df['content'].apply(lambda x: x.lower())
df['content'] = df['content'].apply(lambda x: x.strip())
df['content'] = df['content'].apply(lambda x: x.replace("  ", " "))
df['content'] = df['content'].apply(lambda x: x.replace("\n", ""))

##strip white spaces 
df.columns = df.columns.str.strip()

#lower all the names 
df.colums = df.columns.str.lower()


#filter words that contained in the list 
df['search_word'] = np.where(df['content'].str.contains(search_word),'Yes','No')

##drop duplicate tweets if any 
print(f'The number of tweets before droping duplicates are: {df.shape[0]}')
df = df.drop_duplicates(subset=['date', 'id', 'content', 'username', 'user_location', 'renderedcontent', 'keyword', 'lang', 'url', 'user_id'], keep= 'last')
df.reset_index(drop = True, inplace = True)
print(f'The number of tweets after droping duplicates are: {df.shape[0]}')

#print the data
df.sample(5)

C:\Users\moses\AppData\Local\Temp\ipykernel_15716\3037972859.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].apply(lambda x: x.strftime("%d-%b-%Y %H:%M:%S"))
C:\Users\moses\AppData\Local\Temp\ipykernel_15716\3037972859.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['username'] = df['user'].apply(lambda x: x['username'])
C:\Users\moses\AppData\Local\Temp\ipykernel_15716\3037972859.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

The number of tweets before droping duplicates are: 11
The number of tweets after droping duplicates are: 11


,id,date,content,renderedcontent,replycount,retweetcount,likecount,quotecount,url,lang,...,user_id,user_location,description,followersCount,friendsCount,statusesCount,favouritesCount,protected,mentions,search_word
3,1566832183565815812,05-Sep-2022 00:00:00,@citizentvkenya trojan horse,@citizentvkenya Trojan horse,0,0,1,0,https://twitter.com/Edgerayier/status/15668321...,en,...,173788176,"Nairobi, Kenya",| Architect | Philanthropist | Photography | E...,2162,1478,3771,480,False,[citizentvkenya],Yes
5,1566772313508483072,05-Sep-2022 00:00:00,@nyar_asego @wmmartha supposing he was a troja...,@nyar_asego @WMMartha Supposing he was a Troja...,0,0,0,0,https://twitter.com/chepkotkotbore/status/1566...,en,...,260407108,Nairobi- kenya,,203,313,3011,700,False,"[nyar_asego, WMMartha]",Yes
1,1567214837343698946,06-Sep-2022 00:00:00,@trojan_ragdde @kenyans wewe kacheze huko bana.,@trojan_ragdde @Kenyans Wewe kacheze huko bana.,1,0,0,0,https://twitter.com/Lemeiyan/status/1567214837...,tl,...,901558773864681472,"Nairobi, Kenya",Maasai -We must Preserve Mau Forest,1426,1405,7671,7803,False,"[trojan_ragdde, Kenyans]",Yes
8,1566376384389386243,04-Sep-2022 00:00:00,namimi sjamaliza kusomea the spyware that dele...,namimi sjamaliza kusomea the spyware that dele...,0,0,0,0,https://twitter.com/oneike/status/156637638438...,en,...,555163828,"Nairobi,Kenya","understanding, fierce enterprenuer.\ninterest ...",614,1274,3998,270,False,[],Yes
2,1567074381938974725,06-Sep-2022 00:00:00,@misstichie @marthakarua martha was a trojan h...,@MissTichie @MarthaKarua Martha was a Trojan H...,0,0,0,0,https://twitter.com/TToryquer/status/156707438...,en,...,1455773731423457283,"Nairobi, Kenya",No Bio,541,937,8265,14363,False,"[MissTichie, MarthaKarua]",Yes


### Sample Tweets:

In [316]:
for tweet in df.index[:3]:
    if df['search_word'][tweet] == 'Yes':
        print(f"keyword: {df['keyword'][tweet]} \n Extracted date: {df['date'][tweet]} \n tweet:> {df['renderedcontent'][tweet]} \n link {df['url'][tweet]}")
        print(' '*120)
        print('-'*120)

keyword: Trojan 
 Extracted date: 07-Sep-2022 00:00:00 
 tweet:> @trojan_ragdde @ledamalekina You missed the point.
Tragic again 
 link https://twitter.com/MoffatAkumbi/status/1567440630095265793
                                                                                                                        
------------------------------------------------------------------------------------------------------------------------
keyword: Trojan 
 Extracted date: 06-Sep-2022 00:00:00 
 tweet:> @trojan_ragdde @Kenyans Wewe kacheze huko bana. 
 link https://twitter.com/Lemeiyan/status/1567214837343698946
                                                                                                                        
------------------------------------------------------------------------------------------------------------------------
keyword: Trojan 
 Extracted date: 06-Sep-2022 00:00:00 
 tweet:> @MissTichie @MarthaKarua Martha was a Trojan Horse sent to fuck up Azimio😏I lik

## Tweets containing keywords

In [319]:
df['search_word'].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'

Yes    100.0%
Name: search_word, dtype: object

## Drop the tweets that do not have the keywords

In [320]:
#filter out the only data with matching keyword
data = df[df['search_word']!='No']
data.reset_index(drop = True, inplace = True)
data.sample(3)

,id,date,content,renderedcontent,replycount,retweetcount,likecount,quotecount,url,lang,...,user_id,user_location,description,followersCount,friendsCount,statusesCount,favouritesCount,protected,mentions,search_word
4,1566773694327918596,05-Sep-2022 00:00:00,"now reading conspiracy theories like""uhuru kne...","Now reading conspiracy theories like\n""Uhuru k...",9,7,56,3,https://twitter.com/PolycarpHinga/status/15667...,en,...,480295789,Nairobi,Ambassador of Kinangop to the World| Azimio La...,51827,4710,27822,4448,False,[],Yes
10,1564548121136402433,30-Aug-2022 00:00:00,"tiktok, fb, ig, twtr, other ""social media apps...","TikTok, FB, IG, TWTR, other ""Social Media Apps...",0,0,0,0,https://twitter.com/SimonGako/status/156454812...,en,...,90640725,Nairobi,Entrepreneur.,6076,6636,14373,4913,False,[],Yes
5,1566772313508483072,05-Sep-2022 00:00:00,@nyar_asego @wmmartha supposing he was a troja...,@nyar_asego @WMMartha Supposing he was a Troja...,0,0,0,0,https://twitter.com/chepkotkotbore/status/1566...,en,...,260407108,Nairobi- kenya,,203,313,3011,700,False,"[nyar_asego, WMMartha]",Yes


## Save the File

> Create file Name

In [35]:
#generate a filename 
name = f"african_command_center_tweets"

#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%d_%b").lower()

#extension
extension = ".xlsx"

#generate the name
new_filename = name+"_"+formated_date
old_filename = name+"_"+formated_date+"_"+"oldfile"
print(old_filename ,"\n", new_filename)

african_command_center_tweets_26_aug_oldfile 
 african_command_center_tweets_26_aug


> Write data to Postgresql

In [18]:
#create the connection 

db_host = 'research-dev-db.cluster-c30rc84jwwhk.us-east-2.rds.amazonaws.com'
# db_host = '192.168.8.221'
db_port = 5432
db_password = 'researchpassword'
db = 'postgres'
db_user = 'postgres'


def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db}')

#write the data if the table exists , replace the data
try:
    data.to_sql(new_filename, engine, method=psql_insert_copy, if_exists='replace')
except Exception as e:
    print(f'The error is: {str(e)}')
    pass

> write the african countries cities

In [19]:
#write the data if the table exists , replace the data
table_name = "african_countries_towns_and_cities"

#write the data into db
try:
    coordinates_df.to_sql(table_name, engine, method=psql_insert_copy, if_exists='replace')
except Exception as e:
    print(f'The error is: {str(e)}')
    pass

> Rename file if present in the directory 

In [20]:
#rename the file if is in directory 
try:
  os.rename(old_filename+'.xlsx', new_filename+'xlsx')
except Exception as e:
  print(f"The error is: {str(e)}")
  pass

The error is: [WinError 2] The system cannot find the file specified: 'african_command_center_tweets_26_aug_oldfile.xlsx' -> 'african_command_center_tweets_26_augxlsx'


> Save the File

In [21]:
#write the file
try:
    data.to_excel(new_filename+'.xlsx', sheet_name = 'tweets data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
except BaseException as e:
    print('-'*20)
    print(f"The error is {str(e)}")
    pass

In [22]:
# #write the file
# try:
# #     data.to_excel(new_filename+'.xlsx', sheet_name = 'tweets data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
# except BaseException as e:
#     print('-'*20)
#     print(f"The error is {str(e)}")
#     pass

## Time Spend

In [23]:
#get the time used to proccess the data
end_time = datetime.now()
duration = end_time - start_time
print(f"The stopped at: {end_time} '\n' Started at: {end_time} '\n' and took {duration} to run")

The stopped at: 2022-08-26 04:34:45.556083 '
' Started at: 2022-08-26 04:34:45.556083 '
' and took 2 days, 7:47:54.263342 to run
